In [ ]:
import threading
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


columns = ["Website",'test_name', 'HTTP status code', "Success", "Redirected to ISP Payload"]
df = pd.DataFrame(columns=columns)

text_file = "../../BlockingLists/russianWebsites.txt"


cyta_payload = "This site can’t be reached due tο compliance the Council Regulation (EU) 350/2022 and with EU and National Laws, only for as long as necessary."


def find_payload(text):
   if cyta_payload in text:
       return True
   else: 
       return False

In [184]:
import requests
import socket
import ssl

html_file_with_results = "result.html"
txt_file_with_results = "result.txt"
logs_file = "result.log"


def write_to_file(filename, data):
    with open(filename, "w") as f:
        f.write("%s\n" % data)


def _decode_bytes(input_bytes):
    return input_bytes.decode(errors="replace")


def send_request(
        host, port, data, html_file, txt_file,
        fragment_size=0, fragment_count=0, is_https=True):
    sock = socket.create_connection((host, port), 10)
    if is_https:
        context = ssl.create_default_context()
        sock = context.wrap_socket(sock, server_hostname=host)

    if fragment_count:
        sock.setsockopt(socket.IPPROTO_TCP, socket.TCP_NODELAY, True)

    try:
        for fragment in range(fragment_count):
            write_to_file(html_file, "fragment" + str(fragment))
            write_to_file(txt_file, "fragment" + str(fragment))

            sock.sendall(data[:fragment_size].encode())
            data = data[fragment_size:]

            write_to_file(html_file, str(data))
            write_to_file(txt_file, str(data))

        sock.sendall(data.encode())
        recvdata = sock.recv(8192)
        recv = recvdata
        recv_decoded = recv.decode()
        # print(recv_decoded)
        write_to_file(logs_file, recv_decoded)
        write_to_file(html_file, recv_decoded)
        write_to_file(txt_file, recv_decoded)
        while recvdata:
            recvdata = sock.recv(8192)
            recv += recvdata
    finally:
        try:
            sock.shutdown(socket.SHUT_RDWR)
        except  Exception:
            print("Exception occurred")
        sock.close()
    return _decode_bytes(recv)


def configure_request_body(host, urn="/"):
    requests_body = {
        "normal one ": {
            "data": "GET {} HTTP/1.1\r\n".format(urn)
                    + "Host: {}\r\nConnection: close\r\n\r\n".format(host),
            "fragment_size": 0,
            "fragment_count": 0,
        },
        "extra space after  GET": {
            "data": "GET  {} HTTP/1.0\r\n".format(urn)
                    + "Host: {}\r\nConnection: close\r\n\r\n".format(host),
            "fragment_size": 0,
            "fragment_count": 0,
        },
        "extra line break after GET": {
            "data": "\r\nGET {} HTTP/1.0\r\n".format(urn)
                    + "Host: {}\r\nConnection: close\r\n\r\n".format(host),
            "fragment_size": 0,
            "fragment_count": 0,
        },
        "tab sign after hostname": {
            "data": "GET {} HTTP/1.0\r\n".format(urn)
                    + "Host: {}\t\r\nConnection: close\r\n\r\n".format(host),
            "fragment_size": 0,
            "fragment_count": 0,
        },
        "fragmented header": {
            "data": "GET {} HTTP/1.0\r\n".format(urn)
                    + "Host: {}\r\nConnection: close\r\n\r\n".format(host),
            "fragment_size": 2,
            "fragment_count": 6,
        },
        "dot after hostname": {
            "data": "GET {} HTTP/1.0\r\n".format(urn)
                    + "Host: {}.\r\nConnection: close\r\n\r\n".format(host),
            "fragment_size": 0,
            "fragment_count": 0,
        },
        " hoSt instead Host": {
            "data": "GET {} HTTP/1.0\r\n".format(urn)
                    + "hoSt: {}\r\nConnection: close\r\n\r\n".format(host),
            "fragment_size": 0,
            "fragment_count": 0,
        },
        " hOSt instead Host": {
            "data": "GET {} HTTP/1.0\r\n".format(urn)
                    + "hOSt: {}\r\nConnection: close\r\n\r\n".format(host),
            "fragment_size": 0,
            "fragment_count": 0,
        },
        " Hostname with uppercase": {
            "data": "GET {} HTTP/1.0\r\n".format(urn)
                    + "Host: {}\r\nConnection: close\r\n\r\n".format(host.upper()),
            "fragment_size": 0,
            "fragment_count": 0,
        },
        "missing space sign after Host:": {
            "data": "GET {} HTTP/1.0\r\n".format(urn)
                    + "Host:{}\r\nConnection: close\r\n\r\n".format(host),
            "fragment_size": 0,
            "fragment_count": 0,
        },
        "unix style line break in header": {
            "data": "GET {} HTTP/1.0\n".format(urn)
                    + "Host: {}\nConnection: close\n\n".format(host),
            "fragment_size": 0,
            "fragment_count": 0,
        },
        "unusual order of connection and hostname": {
            "data": "GET {} HTTP/1.0\r\n".format(urn)
                    + "Connection: close\r\nHost: {}\r\n\r\n".format(host),
            "fragment_size": 0,
            "fragment_count": 0,
        },
        "fragmented header, hoSt and missed space": {
            "data": "GET {} HTTP/1.0\r\n".format(urn)
                    + "hoSt:{}\r\nConnection: close\r\n\r\n".format(host),
            "fragment_size": 2,
            "fragment_count": 6,
        },
    }
    return requests_body


def test_dpi(site, port, html_file=html_file_with_results,
             txt_file=txt_file_with_results, is_https=False):
    results = []
    configured_requests = configure_request_body(site, "/")
    for test_name in sorted(configured_requests):
        test = configured_requests[test_name]
        try:
            result = send_request(
                site,
                port,
                test.get("data"),
                html_file,
                txt_file,
                test.get("fragment_size"),
                test.get("fragment_count"),
                is_https,
            )
            df.loc[len(df)] = {
                "Website": site,
                "test_name": test_name,
                "HTTP status code": result.split("\n")[0],
                "Success": result.split("\n")[0].find("200 ") != -1,
                "Redirected to ISP Payload": find_payload(result)
            }
            df.reset_index(drop=True)
            
            # for i in range(len(result.split("\n"))):
            #     print(f"fdsfsdf: {i}", result.split("\n")[i])


        except Exception as e:
            print("🤬 error: ", repr(e))
        else:
            if result.split("\n")[0].find("200 ") != -1:
                print("😘 open successfully")
                results.append((test_name,result))
            else:
                print("😒 can't open")
    return list(set(results))


def trucate_https(domain):
    domain = domain.replace("https://", "")
    domain = domain.replace("http://", "")
    domain = domain.replace("www.", "")
    if domain.endswith("/"):
        domain = domain[:-1]
    return domain


def get_redirection(url, timeout=3):
    try:
            response = requests.get(f"http://{url}", allow_redirects=False)
    except requests.exceptions.ConnectTimeout:
            print("Connection timed out while trying to reach the server.")
            return None, False
    max_redirects = 10
    num_redirects = 0
    redirect_url = f"http://{url}"
    is_https = False
    while response.is_redirect or response.is_permanent_redirect:
        if num_redirects >= max_redirects:
            print("Reached maximum number of redirects.")
            break

        redirect_url = response.headers['Location']
        if redirect_url.find("https") != -1:
            is_https = True
        print(f"Redirecting to: {redirect_url}")
        try:
            response = requests.get(redirect_url, allow_redirects=False, timeout=timeout)
        except requests.exceptions.ConnectTimeout:
            print("Connection timed out while trying to reach the server.")
            return None, False
        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")
            return None, False
        num_redirects += 1
    redirect_url = trucate_https(redirect_url)
    print(redirect_url)
    return redirect_url, is_https


def test_http_or_https(site):
    site, is_https_required = get_redirection(site)
    if is_https_required:
        port = 443
    else:
        port = 80
    return test_dpi(site, port=port, is_https=is_https_required)




In [ ]:

def parse_http_response(response_text):
    headers = {}
    lines = response_text.split('\n')
    status_line = lines[0].strip()
    headers['HTTP Status'] = status_line

    for line in lines[1:]:
        line = line.strip()
        if not line:
            break
        if ':' in line:
            key, value = line.split(':', 1)
            headers[key.strip()] = value.strip()
        else:
            headers[line] = None

    return headers


def test_sites_from_file(filename):
    with open(filename, "r") as file:
        websites = [line.strip() for line in file]

    threads = []
    for site in websites:
        thread = threading.Thread(target=process_site, args=(site,))
        thread.start()
        threads.append(thread)
    for thread in threads:
        thread.join()


def process_site(site):
    test_http_or_https(site)


test_sites_from_file(text_file)     

Redirecting to: https://gosuslugi.ru/
Redirecting to: https://vk.com/
Redirecting to: https://lenta.ru/
Redirecting to: https://iz.ru/
Redirecting to: https://svpressa.ru/
Redirecting to: https://rt.com/
Redirecting to: https://regnum.ru/
Redirecting to: https://vz.ru/
Redirecting to: https://www.rt.ru/
Redirecting to: http://tass.ru/
Redirecting to: https://rg.ru
Redirecting to: https://admburla.ru/
Redirecting to: http://tass.ru/
Redirecting to: https://m.vk.com/
gosuslugi.ru
Redirecting to: https://www.rt.com/
svpressa.ru
iz.ru
rt.ru


Exception in thread Thread-37 (process_site):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/dmitrii/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_44429/3164626645.py", line 34, in process_site
  File "/tmp/ipykernel_44429/866906824.py", line 234, in test_http_or_https
  File "/tmp/ipykernel_44429/866906824.py", line 148, in test_dpi
  File "/tmp/ipykernel_44429/866906824.py", line 113, in configure_request_body
AttributeError: 'NoneType' object has no attribute 'upper'


Redirecting to: http://tass.ru/
vz.ru
An error occurred: HTTPSConnectionPool(host='admburla.ru', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1007)')))
regnum.ru
rg.ru
lenta.ru
😘 open successfully
Redirecting to: https://russian.rt.com/
Redirecting to: http://tass.ru/
Redirecting to: https://www.vesti.ru/
😒 can't open
m.vk.com
😒 can't open
rt.com
😘 open successfully
😘 open successfully
Redirecting to: http://tass.ru/
russian.rt.com
😒 can't open
😒 can't open
😒 can't open
vesti.ru
😘 open successfully
Redirecting to: http://tass.ru/
😘 open successfully😘 open successfully

😘 open successfully
😘 open successfully
Redirecting to: http://tass.ru/
😘 open successfully
😒 can't open
😘 open successfully
😒 can't open
😒 can't open
😒 can't open
Redirecting to: http://tass.ru/
😘 open successfully
😘 open successfully
Redirecting to: http://tass

Exception in thread Thread-26 (process_site):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
Exception in thread Thread-38 (process_site):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
Exception in thread Thread-39 (process_site):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/dmitrii/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    self.run()
  File "/home/dmitrii/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    self.run()
  File "/home/dmitrii/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.10/threading.py", line 953, in run
    _threading_Thread_run(self)
  File "/usr/lib/python3.10/threading.py", li

Connection timed out while trying to reach the server.Connection timed out while trying to reach the server.

Connection timed out while trying to reach the server.


In [ ]:
# df

In [ ]:
df.head()

In [ ]:
df['test_name'].hist()
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
df['HTTP status code'].value_counts().plot(kind='pie', autopct='%1.1f%%',  colors=['blue', 'green', 'coral', 'lightskyblue', 'lightpink'])
plt.title('Distribution of Column1')
plt.ylabel('')
plt.show()



In [ ]:
plt.figure(figsize=(15, 8))

for i, column in enumerate(df.columns):
    plt.subplot(2, 3, i+1)
    if df[column].dtype == 'object':
        sns.countplot(data=df, y=column, order=df[column].value_counts().index)
    else:
        sns.histplot(data=df, x=column)
    plt.title(f'{column} Distribution')
    plt.xlabel('Count' if df[column].dtype == 'object' else 'Value')

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

website_status_counts = df.groupby(['Website', 'HTTP status code']).size().unstack(fill_value=0)

colors = sns.color_palette('bright', len(website_status_counts.columns))

plt.figure(figsize=(15, 10))
for http_status_code, data, color in zip(website_status_counts.columns, website_status_counts.values.T, colors):
    sns.barplot(x=data, y=website_status_counts.index, color=color, label=http_status_code)

plt.title('HTTP Status Code Distribution for Websites', fontsize=20)
plt.xlabel('Count', fontsize=16)
plt.ylabel('Website', fontsize=16)
plt.legend(title='HTTP Status Code', title_fontsize='large', fontsize='large', bbox_to_anchor=(1.05, 1), loc='upper left', labelspacing=1.5)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()

plt.savefig('http_status_code_distribution.png', dpi=300)
plt.show()


In [ ]:
grouped_test_names = df.groupby('test_name')['HTTP status code'].value_counts()
top5_status_codes_names = grouped_test_names.groupby(level=0).nlargest(5).reset_index(level=0, drop=True)

num_tests = len(top5_status_codes_names.index.unique())
rows = (num_tests + 2) // 3
cols = min(num_tests, 3)

for i, (test_name, data) in enumerate(top5_status_codes_names.groupby(level=0)):
    plt.figure(figsize=(10, 10))
    ax = data.plot(kind='bar', legend=False)
    plt.title(f'{test_name}')
    plt.ylabel('Count')
    plt.xticks(rotation=360)
    plt.xlabel('HTTP Status Code')
    plt.tight_layout()
    labels = [f'{code}' for code, description in zip(data.index.get_level_values(1), ['Ok', 'Not Found', 'Redirect', 'Server Error', 'Other'])]
    ax.set_xticklabels(labels, fontsize=6)

    plt.show()

In [ ]:
num_tests = len(top5_status_codes_names.index.unique())
rows = (num_tests + 2) // 3
cols = min(num_tests, 3)


for i, (test_name, data) in enumerate(top5_status_codes_names.groupby(level=0)):
    plt.figure(figsize=(8, 8))
    ax = data.plot(kind='pie', autopct='%1.1f%%', colors=['blue', 'green', 'coral', 'lightskyblue', 'lightpink'],)
    plt.legend(labels=data.index.get_level_values(1), title='HTTP Status Code', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.ylabel('')
    plt.title(f'Distribution of HTTP Status Codes for {test_name}')
    plt.show()